In [1]:
from __future__ import print_function, absolute_import, division
from collections import namedtuple
from cityscapesscripts.helpers.labels import *
import Helper.Helper_functions as hf
import torchvision.transforms.functional as F
from torchvision.utils import draw_segmentation_masks
from PIL import Image
from Helper.ml_models import *
import os
import numpy as np

In [2]:
print("List of cityscapes labels:")
print("")
print("    {:>21} | {:>3} | {:>7} | {:>14} | {:>10} | {:>12} | {:>12} | {:>14}".format('name', 'id', 'trainId', 'category',
                                                                              'categoryId', 'hasInstances',
                                                                              'color', 'ignoreInEval'))
print("    " + ('-' * 98))
counter = 0
for label in labels:
    #if label.ignoreInEval ==0:
    counter +=1
    print("     {:>21} | {:>3} | {:>7} | {:>14} | {:>10} | {:>12} | {:>12} | {}".format(label.name, label.id, label.trainId,
                                                                                  label.category, label.categoryId,
                                                                                  label.hasInstances,
                                                                                  str(label.color),  # Convert the tuple to a string
                                                                                  label.ignoreInEval,
                                                                                  ))
print(" ", '\n', counter)

List of cityscapes labels:

                     name |  id | trainId |       category | categoryId | hasInstances |        color |   ignoreInEval
    --------------------------------------------------------------------------------------------------
                 unlabeled |   0 |     255 |           void |          0 |            0 |    (0, 0, 0) | True
               ego vehicle |   1 |     255 |           void |          0 |            0 |    (0, 0, 0) | True
      rectification border |   2 |     255 |           void |          0 |            0 |    (0, 0, 0) | True
                out of roi |   3 |     255 |           void |          0 |            0 |    (0, 0, 0) | True
                    static |   4 |     255 |           void |          0 |            0 |    (0, 0, 0) | True
                   dynamic |   5 |     255 |           void |          0 |            0 | (111, 74, 0) | True
                    ground |   6 |     255 |           void |          0 |            0 | 

# Create dictionary with the ID and Values for remapping the annotation images

New dictionary gets compared to the cityscapes mapping and differences are printed. Values of 255 got mapped to value 19, so only IDs which initially had the value 155 and got changed to 19 should be printed. 

In [3]:
id_to_trainId = {label.id: label.trainId for label in labels if label.trainId >= 0}
id_to_trainId_updated = {k: 19 if v == 255 else v for k, v in id_to_trainId.items()}

# Iterate over the keys in id_to_trainId_updated
for key in id_to_trainId_updated:
    # Check if the value in id_to_trainId_updated matches the value in id_to_trainId
    if id_to_trainId_updated[key] != id_to_trainId.get(key, None):
        # If the values do not match, print out the key and the corresponding values
        print(f'Key: {key}, id_to_trainId: {id_to_trainId.get(key, None)}, id_to_trainId_updated: {id_to_trainId_updated[key]}')

Key: 0, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 1, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 2, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 3, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 4, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 5, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 6, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 9, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 10, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 14, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 15, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 16, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 18, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 29, id_to_trainId: 255, id_to_trainId_updated: 19
Key: 30, id_to_trainId: 255, id_to_trainId_updated: 19


# Labeling of the Annotations

The annotations get labeled according to the above table, using trainID for the resulting value in the annotation. Annotations with value 255 get labeled to channel 19. All labels got preprocessed by the cityscapes function and sorted intp the CityscapesDaten/semantic_default folder. Here the annotations are still labeled by the id range. Annotations get opened from this folder, processed by a script in this jupyter notebook and saved the CityscapesDaten/semantic folder. 


In [4]:
def create_ground_truth_V2(in_path, out_path, batch_size=6):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create a mapping from label ID to trainId
    id_to_trainId = {label.id: label.trainId for label in labels if label.trainId >= 0}
    id_to_trainId = {k: 19 if v == 255 else v for k, v in id_to_trainId.items()}
    id_to_trainId = torch.tensor([id_to_trainId.get(i, -1) for i in range(256)], dtype=torch.long, device=device)

    # Get the list of image files
    image_files = os.listdir(in_path)

    # Process the images in batches
    for i in tqdm(range(0, len(image_files), batch_size), desc="Processing images"):
        batch_files = image_files[i:i+batch_size]
        batch_images = []
        for image_file in batch_files:
            # Load the annotation image
            annotation = Image.open(os.path.join(in_path, image_file))
            # Convert the annotation image to a PyTorch tensor and move it to the device
            annotation = torch.from_numpy(np.array(annotation)).to(device)
            batch_images.append(annotation)

        # Stack images into a batch
        batch_images = torch.stack(batch_images)

        # Convert the label IDs to trainId
        batch_images = id_to_trainId[batch_images.long()]

        # Save the images
        for j, image in enumerate(batch_images):
            # Convert the tensor back to a PIL image
            image = transforms.ToPILImage()(image.cpu().byte())
            # Save the annotation image with the original name
            image.save(os.path.join(out_path, batch_files[j]))

In [5]:
annotation_input = 'CityscapesDaten/semantic_default'
annotation_output = 'CityscapesDaten/semantic'

os.makedirs(annotation_output, exist_ok=True)

run = False
while run: 
    create_ground_truth_V2(annotation_input, annotation_output)
    break

# Use Dataloader to load an image and annotation from the folder

The shape of the Image should be [3, 520, 520] and the shape of the annotation should be [20, 520, 520]. The unique annotation value range should be
[  0   1   2   5   6   7   8   9  10  11  13  15  18 19] and the Dataloader annotation Value range shoudl be [0. 1.].

In [6]:
annotation_dir = 'CityscapesDaten/semantic'
dataset = CustomDataSet(image_dir='CityscapesDaten/images',
                        annotation_dir='CityscapesDaten/semantic')
image, annotation = dataset.__getitem__(0)

print(f'Image shape: {image.shape}, Annotation shape: {annotation.shape}')
print(f'Dataloader annotation Value range: {np.unique(annotation)}')

annotation_files = os.listdir(annotation_dir)[:100]  # Get the first 100 files

all_values = []

for annotation_file in annotation_files:
    # Load the annotation image
    annotation = Image.open(os.path.join(annotation_dir, annotation_file))
    # Convert the annotation image to a numpy array
    annotation = np.array(annotation)
    # Append the unique values in this image to the list of all values
    all_values.extend(np.unique(annotation))

# Get unique values from all_values
unique_values = np.unique(all_values)

# Print all unique values
print(f'All unique annotation values: {unique_values}')

Image shape: torch.Size([3, 520, 520]), Annotation shape: torch.Size([20, 520, 520])
Dataloader annotation Value range: [0. 1.]
All unique annotation values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
